## <center> RecSys. Home Assignment 1</center>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Один из важных навыков для построения рекомендательных систем - это умение корректно считать метрики качества ранжирования.

В этой домашке вам нужно потренироваться в этом и имплементировать метрики Precision@k, Recall@k, MNAP@k и NDCG@k по формулам, чтобы дальше переиспользовать при построении своих рекомендательных моделей.

Критерии оценивания:
* Что-то пытался сделать, дописал свой код, но ничего не получилось - 1 балл.
* Не совсем корректная имплементация одной из 4 метрик, прохождение части тестов - 1 балл.
* Корректная имплементация одной из 4 метрик, прохождение всех тестов - 2 балла.
* +1 балл, если получится написать Precision@k, Recall@k без циклов.
* +1 балл, если получится написать NDCG@k, MNAP@k без циклов.

Дедлайн сдачи - **24 октября 23:59**.

Формат сдачи - отправить Jupyter notebook на почту ananyeva.me@gmail.com с темой письма "[RecSys HW1]" и названием файла [Name_Surname]_HW1.ipynb.  

Удачи!

In [2]:
import os
os.getcwd()

'/content'

In [3]:
path = '/content/drive/MyDrive/FTIAD_2/RecSys/'
os.chdir(path)
os.getcwd()

'/content/drive/MyDrive/FTIAD_2/RecSys'

In [4]:
from typing import NamedTuple, Union
import tests
import torch

In [5]:
class PrepareTargetResult(NamedTuple):
    values: torch.Tensor
    indices: torch.Tensor


def validate_metric_inputs(output: torch.Tensor, target: torch.Tensor) -> None:
    if output.size() != target.size():
        raise IndexError(
            "Unequal sizes for output and target: "
            f"output - {output.size()}, target - {target.size()}."
        )
    if not (target.eq(0) | target.eq(1)).all():
        raise ValueError(
            "Target contains values outside of 0 and 1." f"\nTarget:\n{target}"
        )


def prepare_target(
    output: torch.Tensor, target: torch.Tensor, return_indices: bool = False
) -> Union[torch.Tensor, PrepareTargetResult]:
    validate_metric_inputs(output, target)
    # Define order by sorted output scores.
    indices = output.argsort(dim=-1, descending=True)
    sorted_target = torch.gather(target, index=indices, dim=-1)
    return (
        PrepareTargetResult(sorted_target, indices) if return_indices else sorted_target
    )


def nan_to_num(tensor: torch.Tensor, nan: float = 0.0) -> torch.Tensor:
    return torch.where(
        torch.isnan(tensor) | torch.isinf(tensor),
        torch.full_like(tensor, fill_value=nan),
        tensor,
    )

# Precision

$$P@k = \frac{1}{k} \sum_{i=1}^k [y_{i} = 1]$$

In [6]:
def precision(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    '''
    output, target ~ (users, items)
    target_sorted_by_output ~ (users, items)
    '''
    target_sorted_by_output = prepare_target(output, target)
    topk = min(target.shape[1], topk)
    sum_precision_k = torch.sum(target_sorted_by_output[:, :topk], 1)
    precision_k = torch.div(sum_precision_k, topk).mean()
    return precision_k

In [7]:
tests.run_precision(precision)

# Recall

$$R@k = \frac{\sum_{i=1}^k [y_{i} = 1]}{\sum_{i=1}^N [y_{i} = 1]}$$

In [8]:
def recall(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    '''
    output, target ~ (users, items)
    target_sorted_by_output ~ (users, items)
    '''
    target_sorted_by_output = prepare_target(output, target)
    numerator = torch.sum(target_sorted_by_output[:, :topk], dim=1)
    denominator = torch.sum(target_sorted_by_output, dim=1)
    recall_k = torch.div(numerator, denominator).mean().nan_to_num()
    return recall_k

In [9]:
tests.run_recall(recall)

# Mean (Normalized) Average Precision


$$AP@k = \sum_{i=1}^{k} \frac{y_{i}}{\sum_{j=1}^{k} \cdot y_{j}} p@i$$
$$MNAP@k = \frac{1}{U} \sum_{u \in U} \frac{1}{min(k, n_u)} AP@k_{u},$$

где $n_u$ - количество айтемов с интеракциями у пользователя $u$ в тестовый период, <br>
$U$ - количество пользователей.

In [10]:
def mnap(output: torch.Tensor, target: torch.Tensor, topk: int, normalized: bool = True) -> torch.Tensor:
    '''
    output, target ~ (users, items)
    target_sorted_by_output ~ (users, items)
    '''
    target_sorted_by_output = prepare_target(output, target)
    relevant_tensors = target_sorted_by_output[:, :topk]
    numerator = relevant_tensors.where((relevant_tensors == 0),
                                       relevant_tensors.cumsum(dim=1))
    denominator = torch.ones(relevant_tensors.shape).cumsum(dim=1)
    if normalized:
        denominator_const = torch.min(torch.tensor([float(topk)] * target.shape[0]),
                                      target.sum(1))
        preMAP_k = torch.div(numerator, denominator).sum(dim=1)
        MAP_k = torch.div(preMAP_k, denominator_const).mean().nan_to_num()
    else:
        MAP_k = torch.div(numerator, denominator).mean(dim=1).mean().nan_to_num()
    return MAP_k

In [11]:
tests.run_map(mnap)
tests.run_mnap(mnap)

AssertionError: ignored

# Normalized Dicsounted Cumulative Gain


$$ NDCG @k = \frac{DCG@k}{IDCG@k},$$ где
$$DCG@k = \sum_{i=1}^{k} \frac{2^{y_{i}} - 1}{log_2 (i + 1)}$$
$$IDCG@k = \sum_{i=1}^{k} \frac{1}{log_2 (i + 1)}$$

In [24]:
def dcg(tensor: torch.Tensor) -> torch.Tensor:
    gains = (2**tensor) - 1
    return gains / torch.log2(torch.arange(0, tensor.size(-1), dtype=torch.float, device=tensor.device) + 2.0)


def ndcg(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    '''
    output, target ~ (users, items)
    target_sorted_by_output ~ (users, items)
    '''
    target_sorted_by_output = prepare_target(output, target)
    ideal_target = prepare_target(target, target)
    numerator = dcg(target_sorted_by_output[:, :topk]).sum(-1).nan_to_num()
    denominator = dcg(ideal_target[:, :topk]).sum(-1).nan_to_num()
    #print('numerator', numerator)
    #print('denominator', denominator)
    #print('torch.div(numerator, denominator)', torch.div(numerator, denominator))
    ndcg = torch.div(numerator, denominator).mean().nan_to_num()
    return ndcg

In [25]:
tests.run_ndcg(ndcg)

AssertionError: ignored

In [ ]:
import math
[(1 / math.log2(2 + 1) + 1 / math.log2(3 + 1) + 1 / math.log2(6 + 1) + 1 / math.log2(7 + 1) + 1 / math.log2(8 + 1) + 1 / math.log2(16 + 1) + 1 / math.log2(19 + 1) + 1 / math.log2(20 + 1)) / (1 / math.log2(1 + 1) + 1 / math.log2(2 + 1) + 1 / math.log2(3 + 1) + 1 / math.log2(4 + 1) + 1 / math.log2(5 + 1) + 1 / math.log2(6 + 1) + 1 / math.log2(7 + 1) + 1 / math.log2(8 + 1)),
   (1 / math.log2(1 + 1) + 1 / math.log2(9 + 1) + 1 / math.log2(10 + 1) + 1 / math.log2(11 + 1) + 1 / math.log2(12 + 1) + 1 / math.log2(13 + 1) + 1 / math.log2(15 + 1) + 1 / math.log2(17 + 1)) / (1 / math.log2(1 + 1) + 1 / math.log2(2 + 1) + 1 / math.log2(3 + 1) + 1 / math.log2(4 + 1) + 1 / math.log2(5 + 1) + 1 / math.log2(6 + 1) + 1 / math.log2(7 + 1) + 1 / math.log2(8 + 1)),
   (1 / math.log2(2 + 1) + 1 / math.log2(4 + 1) + 1 / math.log2(6 + 1) + 1 / math.log2(8 + 1) + 1 / math.log2(9 + 1) + 1 / math.log2(13 + 1) + 1 / math.log2(14 + 1) + 1 / math.log2(16 + 1) + 1 / math.log2(18 + 1) + 1 / math.log2(19 + 1)) / (1 / math.log2(1 + 1) + 1 / math.log2(2 + 1) + 1 / math.log2(3 + 1) + 1 / math.log2(4 + 1) + 1 / math.log2(5 + 1) + 1 / math.log2(6 + 1) + 1 / math.log2(7 + 1) + 1 / math.log2(8 + 1) + 1 / math.log2(9 + 1) + 1 / math.log2(10 + 1)),
   (1 / math.log2(3 + 1) + 1 / math.log2(5 + 1) + 1 / math.log2(8 + 1) + 1 / math.log2(12 + 1) + 1 / math.log2(15 + 1) + 1 / math.log2(16 + 1) + 1 / math.log2(17 + 1) + 1 / math.log2(18 + 1) + 1 / math.log2(19 + 1)) / (1 / math.log2(1 + 1) + 1 / math.log2(2 + 1) + 1 / math.log2(3 + 1) + 1 / math.log2(4 + 1) + 1 / math.log2(5 + 1) + 1 / math.log2(6 + 1) + 1 / math.log2(7 + 1) + 1 / math.log2(8 + 1) + 1 / math.log2(9 + 1))
]

[0.7182647379797555, 0.7314440061077209, 0.718457349578196, 0.6284661941067289]

Долго копался в возможных причинах непрохождения самого-самого последнего теста, но так и не выяснил причину :(((

Очень обидно терять целый балл из-за 1 теста из 24, когда в нём разница меньше одной тысячной...